In [860]:
import pandas as pd
import numpy as np

# Data Preprocessing and Feature Engineering

In [863]:
# Function to split a list into chunks of 3 elements
def split_into_chunks(lst):
    # Create an empty list to store the chunks
    chunks = []
    
    # Loop through the list in steps of chunk_size
    for i in range(0, len(lst), 3):
        # Slice the list from index i to i + chunk_size and append to chunks
        chunk = lst[i:i + 3]
        chunks.append(chunk)
    
    # Return the list of chunks
    return chunks

In [865]:
w_data_list = []

# Read the files sequentially by dynamically generating file paths
for i in range(1, 14):
    weather_path = f"ML-Proj-Dataset\\Weather\\{i}.xlsx"   # Generate path for each file
    w_data_list.append(pd.read_excel(weather_path))

# Concatenate all the data into one DataFrame
w_data = pd.concat(w_data_list, ignore_index=True)
# Convert all string columns to lowercase
w_data = w_data.apply(lambda x: x.str.lower() if x.dtype == "object" else x)

# Now, try removing duplicates
w_data = w_data.drop_duplicates()

In [866]:
w_data['Temperature (°F)'] = w_data['Temperature (°F)'].apply(lambda x: x.split(' '))
w_data['Dew Point (°F)'] = w_data['Dew Point (°F)'].apply(lambda x: x.split(' '))
w_data['Humidity (%)'] = w_data['Humidity (%)'].apply(lambda x: x.split(' '))
w_data['Wind Speed (mph)'] = w_data['Wind Speed (mph)'].apply(lambda x: x.split(' '))
w_data['Pressure (in)'] = w_data['Pressure (in)'].apply(lambda x: x.split(' '))
w_data['Precipitation (in)'] = w_data['Precipitation (in)'].str.split()

In [869]:
# Remove the first three non-numeric values (Max, Avg, Min)
w_data['Temperature (°F)'] = w_data['Temperature (°F)'].apply(lambda x: x[3:])
w_data['Dew Point (°F)'] = w_data['Dew Point (°F)'].apply(lambda x: x[3:])
w_data['Humidity (%)'] = w_data['Humidity (%)'].apply(lambda x: x[3:])
w_data['Wind Speed (mph)'] = w_data['Wind Speed (mph)'].apply(lambda x: x[3:])
w_data['Pressure (in)'] = w_data['Pressure (in)'].apply(lambda x: x[3:])
w_data['Precipitation (in)'] = w_data['Precipitation (in)'].apply(lambda x: x[1:])

In [871]:
# w_data = w_data.explode('Date')
#Apply this function to each list in the 'Temperature (°F)' column
w_data['Temperature (°F)'] = w_data['Temperature (°F)'].apply(lambda x: split_into_chunks(x))
w_data['Dew Point (°F)'] = w_data['Dew Point (°F)'].apply(lambda x: split_into_chunks(x))
w_data['Humidity (%)'] = w_data['Humidity (%)'].apply(lambda x: split_into_chunks(x))
w_data['Wind Speed (mph)'] = w_data['Wind Speed (mph)'].apply(lambda x: split_into_chunks(x))
w_data['Pressure (in)'] = w_data['Pressure (in)'].apply(lambda x: split_into_chunks(x))

In [873]:
# w_data = process_weather_data(w_data)

In [875]:
temp = w_data['Temperature (°F)'].explode().reset_index(drop=True)
dew =  w_data['Dew Point (°F)'].explode().reset_index(drop=True)
hum = w_data['Humidity (%)'].explode().reset_index(drop=True)
wind = w_data['Wind Speed (mph)'].explode().reset_index(drop=True)
press = w_data['Pressure (in)'].explode().reset_index(drop=True)
preci = w_data['Precipitation (in)'].explode().reset_index(drop=True) 

In [877]:
# Assuming 'Time' column contains month and date info like 'Jul 1 2 3 4 5'
w_data[['Month', 'Date']] = w_data['Time'].str.split(' ', expand=True, n=1)

In [879]:
# Drop the original 'Time' column
w_data['Date'] = w_data['Date'].str.split()

# Explode the 'Date' column
w_data = w_data.explode('Date', ignore_index=True)
w_data['Temperature (°F)'] = temp
w_data['Dew Point (°F)'] = dew
w_data['Humidity (%)'] = hum
w_data['Wind Speed (mph)'] = wind
w_data['Pressure (in)'] = press
w_data['Precipitation (in)'] = preci



In [881]:
w_data.drop('Time', axis=1, inplace=True)

In [883]:
w_data['Temperature (°F)'] = temp
w_data['Dew Point (°F)'] = dew
w_data['Humidity (%)'] = hum
w_data['Wind Speed (mph)'] = wind
w_data['Pressure (in)'] = press

In [885]:
def get_year(index):
    # Determine whether it's the first year (2023) or second year (2024)
    if index < len(w_data) // 2:
        return 2023  # First half
    else:
        return 2024  # Second half

# function to assign the correct year
w_data['Year'] = [get_year(idx) for idx in range(len(w_data))]

In [887]:
w_data.columns

Index(['Temperature (°F)', 'Dew Point (°F)', 'Humidity (%)',
       'Wind Speed (mph)', 'Pressure (in)', 'Precipitation (in)', 'Month',
       'Date', 'Year'],
      dtype='object')

In [889]:
w_data['Date'] = w_data['Year'].astype(str) + ' ' + w_data['Month'] + ' ' + w_data['Date']
w_data['Date'] = pd.to_datetime(w_data['Date'], format='%Y %b %d')
w_data['Date'] = w_data['Date'].dt.date
w_data.drop('Month', axis=1, inplace=True)
w_data.drop('Year', axis=1, inplace=True)

In [891]:
columns = ['Temperature (°F)', 'Dew Point (°F)', 'Humidity (%)', 'Wind Speed (mph)', 'Pressure (in)']
for col in columns:
    w_data[col] = w_data[col].apply(lambda x: x[1])


In [893]:
w_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Temperature (°F)    380 non-null    object
 1   Dew Point (°F)      380 non-null    object
 2   Humidity (%)        380 non-null    object
 3   Wind Speed (mph)    380 non-null    object
 4   Pressure (in)       380 non-null    object
 5   Precipitation (in)  380 non-null    object
 6   Date                380 non-null    object
dtypes: object(7)
memory usage: 20.9+ KB


In [626]:
w_data.to_csv('weather_data.csv', index=False)

In [895]:
w_data

,Temperature (°F),Dew Point (°F),Humidity (%),Wind Speed (mph),Pressure (in),Precipitation (in),Date
0,88.8,74.5,63.4,8.9,28.8,0.00,2023-07-01
1,91.5,75.8,61.3,6.7,28.8,0.00,2023-07-02
2,91.7,77.0,63.7,11.0,28.8,0.00,2023-07-03
3,88.5,74.3,64.0,13.0,28.7,0.00,2023-07-04
4,76.0,72.9,90.2,15.0,28.8,0.00,2023-07-05
...,...,...,...,...,...,...,...
375,88.2,76.7,69.8,8.6,28.9,0.00,2024-07-10
376,90.7,76.8,65.7,6.7,28.7,0.00,2024-07-11
377,79.4,74.9,85.5,14.0,28.7,0.00,2024-07-12
378,85.5,71.7,65.2,5.4,28.7,0.00,2024-07-13


In [897]:
import json

In [899]:
import json
from docx import Document
import os

In [90]:

# List to store data from all files

# Function to process files
def process_files(file_type):
    all_data = []
    for i in range(1, 73):  # Update the range if needed
        file_path = f"ML-Proj-Dataset\\{file_type}\\{i}.docx"
        if os.path.exists(file_path):
            doc = Document(file_path)
            data = "\n".join([p.text for p in doc.paragraphs])  # Combine all paragraphs  
            flights_data = json.loads(data)  # Parse the JSON-like text
            all_data.extend(flights_data)  # Add to the list
        else:
            print(f"{file_path} does not exist.")
            # Normalize the data
    df = pd.json_normalize(all_data)
    # Rename the columns to more suitable names
    if (file_type != 'Test'):
        df.columns = [
            "Flight Type", 
            "Status", 
            "Departure IATA Code", 
            "Departure ICAO Code", 
            "Departure Terminal", 
            "Departure Scheduled Time", 
            "Departure Estimated Time", 
            "Departure Actual Time", 
            "Departure Estimated Runway", 
            "Departure Actual Runway", 
            "Arrival IATA Code", 
            "Arrival ICAO Code", 
            "Arrival Terminal", 
            "Arrival Scheduled Time", 
            "Arrival Estimated Time", 
            "Airline Name", 
            "Airline IATA Code", 
            "Airline ICAO Code", 
            "Flight Number", 
            "Flight IATA Number", 
            "Flight ICAO Number", 
            "Arrival Baggage", 
            "Codeshare Airline Name", 
            "Codeshare Airline IATA Code", 
            "Codeshare Airline ICAO Code", 
            "Codeshare Flight Number", 
            "Codeshare Flight IATA Number", 
            "Codeshare Flight ICAO Number", 
            "Arrival Gate", 
            "Departure Gate", 
            "Arrival Actual Time", 
            "Arrival Estimated Runway", 
            "Arrival Actual Runway"
        ]
    else:
           df.columns = [
            "Flight Type", 
            "Status", 
            "Departure IATA Code", 
            "Departure ICAO Code", 
            "Departure Scheduled Time", 
            "Departure Estimated Runway", 
            "Departure Actual Runway", 
            "Arrival IATA Code", 
            "Arrival ICAO Code", 
            "Arrival Scheduled Time", 
            "Arrival Estimated Time", 
            "Airline Name", 
            "Airline IATA Code", 
            "Airline ICAO Code", 
            "Flight Number", 
            "Flight IATA Number", 
            "Flight ICAO Number", 
            "Departure Terminal", 
            "Arrival Terminal", 
            "Arrival Baggage", 
            "Codeshare Airline Name", 
            "Codeshare Airline IATA Code", 
            "Codeshare Airline ICAO Code", 
            "Codeshare Flight Number", 
            "Codeshare Flight IATA Number", 
            "Codeshare Flight ICAO Number", 
            "Departure Gate", 
            "Arrival Gate", 
            "Arrival Actual Time", 
            "Arrival Estimated Runway", 
            "Arrival Actual Runway"
        ]

    return df

        

# Process both Train and Test files
train = process_files("Train")
test = process_files("Test")


# Print a message indicating that all files were processed
print(f"Processed records from both Train and Test datasets.")

ML-Proj-Dataset\Train\38.docx does not exist.
ML-Proj-Dataset\Test\38.docx does not exist.
Processed records from both Train and Test datasets.


In [91]:
# test.head(10)

In [92]:
train.to_excel('train_flight_data.xlsx', index=False)
test.to_excel('test_flight_data.xlsx', index=False)

In [901]:
train = pd.read_excel(r"C:\Users\mahee\Downloads\ML_Project\train_flight_data.xlsx")
test = pd.read_excel(r"C:\Users\mahee\Downloads\ML_Project\test_flight_data.xlsx")
#w_data = pd.read_csv('weather_data.csv')

In [902]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51572 entries, 0 to 51571
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Flight Type                   51572 non-null  object 
 1   Status                        51572 non-null  object 
 2   Departure IATA Code           51572 non-null  object 
 3   Departure ICAO Code           51572 non-null  object 
 4   Departure Terminal            29822 non-null  object 
 5   Departure Scheduled Time      51572 non-null  object 
 6   Departure Estimated Time      44565 non-null  object 
 7   Departure Actual Time         30990 non-null  object 
 8   Departure Estimated Runway    30990 non-null  object 
 9   Departure Actual Runway       30990 non-null  object 
 10  Arrival IATA Code             51572 non-null  object 
 11  Arrival ICAO Code             51572 non-null  object 
 12  Arrival Terminal              20565 non-null  object 
 13  A

In [903]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14910 entries, 0 to 14909
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Flight Type                   14910 non-null  object 
 1   Status                        14910 non-null  object 
 2   Departure IATA Code           14910 non-null  object 
 3   Departure ICAO Code           14910 non-null  object 
 4   Departure Scheduled Time      14910 non-null  object 
 5   Departure Estimated Runway    8695 non-null   object 
 6   Departure Actual Runway       8695 non-null   object 
 7   Arrival IATA Code             14910 non-null  object 
 8   Arrival ICAO Code             14910 non-null  object 
 9   Arrival Scheduled Time        14910 non-null  object 
 10  Arrival Estimated Time        8395 non-null   object 
 11  Airline Name                  14910 non-null  object 
 12  Airline IATA Code             14904 non-null  object 
 13  A

In [904]:
# test_binary.rename(columns={'Type':'Flight Type','Scheduled Time': 'Departure Scheduled Time'}, inplace=True)

In [905]:
# test_binary['Departure Scheduled Time'] = pd.to_datetime(test_binary['Departure Scheduled Time'], errors='coerce')
# test_binary['Departure Scheduled Date'] = test_binary['Departure Scheduled Time'].dt.date
# test_binary['Departure Scheduled Time'] = test_binary['Departure Scheduled Time'].apply(lambda x: x.timestamp())

##### Make them consistent

In [907]:
train_cols = train.columns
test_cols = test.columns
for i in train_cols:
    if i not in test_cols:
        print(i)

Departure Estimated Time
Departure Actual Time


In [908]:
train.drop(columns=['Departure Estimated Time', 'Departure Actual Time'], inplace = True)

In [909]:
train.isnull().sum()


Flight Type                         0
Status                              0
Departure IATA Code                 0
Departure ICAO Code                 0
Departure Terminal              21750
Departure Scheduled Time            0
Departure Estimated Runway      20582
Departure Actual Runway         20582
Arrival IATA Code                   0
Arrival ICAO Code                   0
Arrival Terminal                31007
Arrival Scheduled Time              0
Arrival Estimated Time          21364
Airline Name                        0
Airline IATA Code                  29
Airline ICAO Code                   0
Flight Number                       0
Flight IATA Number                 29
Flight ICAO Number                  0
Arrival Baggage                 33454
Codeshare Airline Name          35689
Codeshare Airline IATA Code     35689
Codeshare Airline ICAO Code     35689
Codeshare Flight Number         35689
Codeshare Flight IATA Number    35689
Codeshare Flight ICAO Number    35689
Arrival Gate

In [910]:
test.isnull().sum()

Flight Type                         0
Status                              0
Departure IATA Code                 0
Departure ICAO Code                 0
Departure Scheduled Time            0
Departure Estimated Runway       6215
Departure Actual Runway          6215
Arrival IATA Code                   0
Arrival ICAO Code                   0
Arrival Scheduled Time              0
Arrival Estimated Time           6515
Airline Name                        0
Airline IATA Code                   6
Airline ICAO Code                   1
Flight Number                       0
Flight IATA Number                  6
Flight ICAO Number                  1
Departure Terminal               6476
Arrival Terminal                 8819
Arrival Baggage                  9629
Codeshare Airline Name          10247
Codeshare Airline IATA Code     10247
Codeshare Airline ICAO Code     10247
Codeshare Flight Number         10247
Codeshare Flight IATA Number    10247
Codeshare Flight ICAO Number    10247
Departure Ga

In [911]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14910 entries, 0 to 14909
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Flight Type                   14910 non-null  object 
 1   Status                        14910 non-null  object 
 2   Departure IATA Code           14910 non-null  object 
 3   Departure ICAO Code           14910 non-null  object 
 4   Departure Scheduled Time      14910 non-null  object 
 5   Departure Estimated Runway    8695 non-null   object 
 6   Departure Actual Runway       8695 non-null   object 
 7   Arrival IATA Code             14910 non-null  object 
 8   Arrival ICAO Code             14910 non-null  object 
 9   Arrival Scheduled Time        14910 non-null  object 
 10  Arrival Estimated Time        8395 non-null   object 
 11  Airline Name                  14910 non-null  object 
 12  Airline IATA Code             14904 non-null  object 
 13  A

In [912]:
drop = ['Departure Terminal', 'Departure Estimated Runway', 'Arrival Terminal', 'Arrival Estimated Time', 'Arrival Baggage', 'Codeshare Airline Name', 'Codeshare Airline IATA Code', 'Codeshare Airline ICAO Code', 'Codeshare Flight Number', 'Codeshare Flight IATA Number', 'Codeshare Flight ICAO Number', 'Arrival Gate', 'Departure Gate', 'Arrival Actual Time', 'Arrival Estimated Runway', 'Arrival Actual Runway']
train.drop(columns = drop, inplace = True)
test.drop(columns = drop, inplace = True)

In [913]:
#train.dropna(subset=['Departure Actual Time'], inplace = True)

In [914]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51572 entries, 0 to 51571
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Flight Type               51572 non-null  object
 1   Status                    51572 non-null  object
 2   Departure IATA Code       51572 non-null  object
 3   Departure ICAO Code       51572 non-null  object
 4   Departure Scheduled Time  51572 non-null  object
 5   Departure Actual Runway   30990 non-null  object
 6   Arrival IATA Code         51572 non-null  object
 7   Arrival ICAO Code         51572 non-null  object
 8   Arrival Scheduled Time    51572 non-null  object
 9   Airline Name              51572 non-null  object
 10  Airline IATA Code         51543 non-null  object
 11  Airline ICAO Code         51572 non-null  object
 12  Flight Number             51572 non-null  int64 
 13  Flight IATA Number        51543 non-null  object
 14  Flight ICAO Number    

In [915]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14910 entries, 0 to 14909
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Flight Type               14910 non-null  object
 1   Status                    14910 non-null  object
 2   Departure IATA Code       14910 non-null  object
 3   Departure ICAO Code       14910 non-null  object
 4   Departure Scheduled Time  14910 non-null  object
 5   Departure Actual Runway   8695 non-null   object
 6   Arrival IATA Code         14910 non-null  object
 7   Arrival ICAO Code         14910 non-null  object
 8   Arrival Scheduled Time    14910 non-null  object
 9   Airline Name              14910 non-null  object
 10  Airline IATA Code         14904 non-null  object
 11  Airline ICAO Code         14909 non-null  object
 12  Flight Number             14910 non-null  int64 
 13  Flight IATA Number        14904 non-null  object
 14  Flight ICAO Number    

In [916]:
def convert_to_datetime(data, file_type):

    # if file_type != 'Test':
    #      data['Departure Actual Time'] = pd.to_datetime(data['Departure Actual Time'], errors='coerce')
         #data['Departure Estimated Time'] = pd.to_datetime(data['Departure Estimated Time'], errors='coerce')
        
    data['Departure Scheduled Time'] = pd.to_datetime(data['Departure Scheduled Time'], errors='coerce')
    data['Arrival Scheduled Time'] = pd.to_datetime(data['Arrival Scheduled Time'], errors='coerce')
    #data['Arrival Actual Time'] = pd.to_datetime(data['Arrival Actual Time'], errors='coerce')
    #data['Arrival Estimated Time'] = pd.to_datetime(data['Arrival Estimated Time'], errors='coerce')
    #data['Departure Estimated Runway'] = pd.to_datetime(data['Departure Estimated Runway'], errors='coerce')
    data['Departure Actual Runway'] = pd.to_datetime(data['Departure Actual Runway'], errors='coerce')
    #data['Arrival Estimated Runway'] = pd.to_datetime(data['Arrival Estimated Runway'], errors='coerce')
    #data['Arrival Actual Runway'] = pd.to_datetime(data['Arrival Actual Runway'], errors='coerce')
    
    return data

train = convert_to_datetime(train, file_type = 'Train')
test = convert_to_datetime(test, file_type = 'Test') 

In [917]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51572 entries, 0 to 51571
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Flight Type               51572 non-null  object        
 1   Status                    51572 non-null  object        
 2   Departure IATA Code       51572 non-null  object        
 3   Departure ICAO Code       51572 non-null  object        
 4   Departure Scheduled Time  51572 non-null  datetime64[ns]
 5   Departure Actual Runway   30990 non-null  datetime64[ns]
 6   Arrival IATA Code         51572 non-null  object        
 7   Arrival ICAO Code         51572 non-null  object        
 8   Arrival Scheduled Time    51572 non-null  datetime64[ns]
 9   Airline Name              51572 non-null  object        
 10  Airline IATA Code         51543 non-null  object        
 11  Airline ICAO Code         51572 non-null  object        
 12  Flight Number     

In [918]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14910 entries, 0 to 14909
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Flight Type               14910 non-null  object        
 1   Status                    14910 non-null  object        
 2   Departure IATA Code       14910 non-null  object        
 3   Departure ICAO Code       14910 non-null  object        
 4   Departure Scheduled Time  14910 non-null  datetime64[ns]
 5   Departure Actual Runway   8695 non-null   datetime64[ns]
 6   Arrival IATA Code         14910 non-null  object        
 7   Arrival ICAO Code         14910 non-null  object        
 8   Arrival Scheduled Time    14910 non-null  datetime64[ns]
 9   Airline Name              14910 non-null  object        
 10  Airline IATA Code         14904 non-null  object        
 11  Airline ICAO Code         14909 non-null  object        
 12  Flight Number     

## Handle Missing Values

In [920]:
train["Delay"] = (train["Departure Actual Runway"] - train["Departure Scheduled Time"]).dt.total_seconds()
test["Delay"] = (test['Departure Actual Runway'] - test["Departure Scheduled Time"]).dt.total_seconds()

In [921]:
#train.info()

In [922]:
def fill_target_cols(data, file_type):
    
    average_delay = data["Delay"].median()

    # if file_type != 'Test':
    
    #     data["Departure Actual Runway"] = data["Departure Actual Runway"].fillna(data["Departure Scheduled Time"] + pd.to_timedelta(average_delay, unit='s'))
    #     #data["Departure Estimated Time"] = data["Departure Estimated Time"].fillna(data["Departure Scheduled Time"] + pd.to_timedelta(average_delay, unit='s') )
        
    #     #data["Departure Actual Runway"] = data["Departure Actual Runway"].fillna(data["Departure Scheduled Time"])
    #     data["Delay"] = (data["Departure Actual Runway"] - data["Departure Scheduled Time"]).dt.total_seconds()

    data["Departure Actual Runway"] = data["Departure Actual Runway"].fillna(data["Departure Scheduled Time"] + pd.to_timedelta(average_delay, unit='s'))
    #data["Departure Actual Runway"] = data["Departure Actual Runway"].fillna(data["Departure Scheduled Time"])
    
    data["Delay"] = (data["Departure Actual Runway"] - data["Departure Scheduled Time"]).dt.total_seconds()

    return data

# Apply the function to train and test datasets
train = fill_target_cols(train, 'Train')
test = fill_target_cols(test, 'Test')



In [923]:
#train["Delay"] = (train["Departure Actual Runway"] - train["Departure Scheduled Time"]).dt.total_seconds()
#test["Delay"] = (test['Departure Actual Runway'] - test["Departure Scheduled Time"]).dt.total_seconds()

In [924]:
#test.info()

In [925]:
def fill_missing_values(data):
    cols = data.columns[data.isnull().any()]
    print(cols)

    for c in cols:
            
        mode_value = data[c].mode()[0]
        data[c] = data[c].fillna(mode_value)
        
    return data

# Apply the function to both train and test data
train = fill_missing_values(train)
test = fill_missing_values(test)


Index(['Airline IATA Code', 'Flight IATA Number'], dtype='object')
Index(['Airline IATA Code', 'Airline ICAO Code', 'Flight IATA Number',
       'Flight ICAO Number'],
      dtype='object')


In [926]:
#train.info()

In [927]:
# train["Delay"] = (train["Departure Actual Time"] - train["Departure Scheduled Time"]).dt.total_seconds()
# test["Delay"] = (test['Departure Actual Runway'] - test["Departure Scheduled Time"]).dt.total_seconds()

In [928]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14910 entries, 0 to 14909
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Flight Type               14910 non-null  object        
 1   Status                    14910 non-null  object        
 2   Departure IATA Code       14910 non-null  object        
 3   Departure ICAO Code       14910 non-null  object        
 4   Departure Scheduled Time  14910 non-null  datetime64[ns]
 5   Departure Actual Runway   14910 non-null  datetime64[ns]
 6   Arrival IATA Code         14910 non-null  object        
 7   Arrival ICAO Code         14910 non-null  object        
 8   Arrival Scheduled Time    14910 non-null  datetime64[ns]
 9   Airline Name              14910 non-null  object        
 10  Airline IATA Code         14910 non-null  object        
 11  Airline ICAO Code         14910 non-null  object        
 12  Flight Number     

In [929]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51572 entries, 0 to 51571
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Flight Type               51572 non-null  object        
 1   Status                    51572 non-null  object        
 2   Departure IATA Code       51572 non-null  object        
 3   Departure ICAO Code       51572 non-null  object        
 4   Departure Scheduled Time  51572 non-null  datetime64[ns]
 5   Departure Actual Runway   51572 non-null  datetime64[ns]
 6   Arrival IATA Code         51572 non-null  object        
 7   Arrival ICAO Code         51572 non-null  object        
 8   Arrival Scheduled Time    51572 non-null  datetime64[ns]
 9   Airline Name              51572 non-null  object        
 10  Airline IATA Code         51572 non-null  object        
 11  Airline ICAO Code         51572 non-null  object        
 12  Flight Number     

## Data Integration

In [931]:
train.columns

Index(['Flight Type', 'Status', 'Departure IATA Code', 'Departure ICAO Code',
       'Departure Scheduled Time', 'Departure Actual Runway',
       'Arrival IATA Code', 'Arrival ICAO Code', 'Arrival Scheduled Time',
       'Airline Name', 'Airline IATA Code', 'Airline ICAO Code',
       'Flight Number', 'Flight IATA Number', 'Flight ICAO Number', 'Delay'],
      dtype='object')

## Data Cleaning and Transformation

In [933]:
test.columns

Index(['Flight Type', 'Status', 'Departure IATA Code', 'Departure ICAO Code',
       'Departure Scheduled Time', 'Departure Actual Runway',
       'Arrival IATA Code', 'Arrival ICAO Code', 'Arrival Scheduled Time',
       'Airline Name', 'Airline IATA Code', 'Airline ICAO Code',
       'Flight Number', 'Flight IATA Number', 'Flight ICAO Number', 'Delay'],
      dtype='object')

## Handle Missing Values

In [935]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51572 entries, 0 to 51571
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Flight Type               51572 non-null  object        
 1   Status                    51572 non-null  object        
 2   Departure IATA Code       51572 non-null  object        
 3   Departure ICAO Code       51572 non-null  object        
 4   Departure Scheduled Time  51572 non-null  datetime64[ns]
 5   Departure Actual Runway   51572 non-null  datetime64[ns]
 6   Arrival IATA Code         51572 non-null  object        
 7   Arrival ICAO Code         51572 non-null  object        
 8   Arrival Scheduled Time    51572 non-null  datetime64[ns]
 9   Airline Name              51572 non-null  object        
 10  Airline IATA Code         51572 non-null  object        
 11  Airline ICAO Code         51572 non-null  object        
 12  Flight Number     

In [936]:
def process_datetime_columns(data, file_type):

    #if file_type != 'Test':
        # data['Departure Actual Day'] = data['Departure Actual Time'].dt.day_name()
        # data['Departure Actual Date'] = data['Departure Actual Time'].dt.date
        # data['Departure Actual Hour'] = data['Departure Actual Time'].dt.hour
        # data.drop(columns = ['Departure Actual Time'], inplace = True)

        #data['Departure Estimated Date'] = data['Departure Estimated Time'].dt.date
        #data['Departure Estimated Time'] = data['Departure Estimated Time'].dt.time

    # Processing 'Departure Scheduled Time'
    data['Departure Scheduled Day'] = data['Departure Scheduled Time'].dt.day_name()
    data['Departure Scheduled Date'] = data['Departure Scheduled Time'].dt.date
    data['Departure Scheduled Hour'] = data['Departure Scheduled Time'].dt.hour
    data['Departure Scheduled Year'] = data['Departure Scheduled Time'].dt.year
    data.drop(columns = ['Departure Scheduled Time'], inplace = True)


    data['Departure Actual Runway Date'] = data['Departure Actual Runway'].dt.date
    data['Departure Actual Runway Day'] = data['Departure Actual Runway'].dt.day_name()
    data['Departure Actual Runway Hour'] = data['Departure Actual Runway'].dt.hour
    data['Departure Actual Runway Year'] = data['Departure Actual Runway'].dt.year
    
    data.drop(columns = ['Departure Actual Runway'], inplace = True)
    

    
    return data

train = process_datetime_columns(train, 'Train')
test = process_datetime_columns(test, 'Test')


In [937]:
# def process_datetime_columns(data, file_type):

#     datetime_columns = ['Departure Scheduled Time','Arrival Scheduled Time','Departure Actual Runway']
        
#     data['Departure Scheduled Date'] = data['Departure Scheduled Time'].dt.date
#     data['Departure Scheduled Time'] = data['Departure Scheduled Time'].apply(lambda x: x.timestamp())
#     data['Arrival Scheduled Time'] = data['Arrival Scheduled Time'].apply(lambda x: x.timestamp())


#     data['Departure Estimated Runway'] = data['Departure Estimated Runway'].apply(lambda x: x.timestamp())
#     data['Departure Actual Runway'] = data['Departure Actual Runway'].apply(lambda x: x.timestamp())
#     # for c in datetime_columns:
#     #         data[c] = data[c].view('int64') // 10**9
    
#     return data

# train = process_datetime_columns(train, 'Train')
# test = process_datetime_columns(test, 'Test')


In [938]:
test.columns

Index(['Flight Type', 'Status', 'Departure IATA Code', 'Departure ICAO Code',
       'Arrival IATA Code', 'Arrival ICAO Code', 'Arrival Scheduled Time',
       'Airline Name', 'Airline IATA Code', 'Airline ICAO Code',
       'Flight Number', 'Flight IATA Number', 'Flight ICAO Number', 'Delay',
       'Departure Scheduled Day', 'Departure Scheduled Date',
       'Departure Scheduled Hour', 'Departure Scheduled Year',
       'Departure Actual Runway Date', 'Departure Actual Runway Day',
       'Departure Actual Runway Hour', 'Departure Actual Runway Year'],
      dtype='object')

In [939]:
w_data['Date'] = pd.to_datetime(w_data['Date'])
train = pd.merge(train, w_data, left_on="Departure Scheduled Date", right_on="Date", how="left")
test = pd.merge(test, w_data, left_on="Departure Scheduled Date", right_on="Date", how="left")
#test_binary = pd.merge(test_binary, w_data, left_on="Departure Scheduled Date", right_on="Date", how="left")

In [940]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51572 entries, 0 to 51571
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Flight Type                   51572 non-null  object        
 1   Status                        51572 non-null  object        
 2   Departure IATA Code           51572 non-null  object        
 3   Departure ICAO Code           51572 non-null  object        
 4   Arrival IATA Code             51572 non-null  object        
 5   Arrival ICAO Code             51572 non-null  object        
 6   Arrival Scheduled Time        51572 non-null  datetime64[ns]
 7   Airline Name                  51572 non-null  object        
 8   Airline IATA Code             51572 non-null  object        
 9   Airline ICAO Code             51572 non-null  object        
 10  Flight Number                 51572 non-null  int64         
 11  Flight IATA Number          

In [941]:
#test.columns

In [942]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14910 entries, 0 to 14909
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Flight Type                   14910 non-null  object        
 1   Status                        14910 non-null  object        
 2   Departure IATA Code           14910 non-null  object        
 3   Departure ICAO Code           14910 non-null  object        
 4   Arrival IATA Code             14910 non-null  object        
 5   Arrival ICAO Code             14910 non-null  object        
 6   Arrival Scheduled Time        14910 non-null  datetime64[ns]
 7   Airline Name                  14910 non-null  object        
 8   Airline IATA Code             14910 non-null  object        
 9   Airline ICAO Code             14910 non-null  object        
 10  Flight Number                 14910 non-null  int64         
 11  Flight IATA Number          

In [943]:
train.drop(columns=['Date'], inplace=True)
test.drop(columns=['Date'], inplace=True)

In [944]:
# test_binary.drop(columns=['Date', 'Departure Scheduled Date'], inplace=True)


In [945]:
# test_binary.drop(columns = ['Arrival Estimated Time'], inplace = True)

In [946]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51572 entries, 0 to 51571
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Flight Type                   51572 non-null  object        
 1   Status                        51572 non-null  object        
 2   Departure IATA Code           51572 non-null  object        
 3   Departure ICAO Code           51572 non-null  object        
 4   Arrival IATA Code             51572 non-null  object        
 5   Arrival ICAO Code             51572 non-null  object        
 6   Arrival Scheduled Time        51572 non-null  datetime64[ns]
 7   Airline Name                  51572 non-null  object        
 8   Airline IATA Code             51572 non-null  object        
 9   Airline ICAO Code             51572 non-null  object        
 10  Flight Number                 51572 non-null  int64         
 11  Flight IATA Number          

In [947]:
columns_to_fill = [
    "Temperature (°F)",
    "Dew Point (°F)",
    "Humidity (%)",
    "Wind Speed (mph)",
    "Pressure (in)",
    "Precipitation (in)"
]

# Apply forward fill to the selected columns
train[columns_to_fill] = train[columns_to_fill].ffill(axis=0)
test[columns_to_fill] = test[columns_to_fill].ffill(axis=0)
#test_binary[columns_to_fill] = test[columns_to_fill].ffill(axis=0)

In [948]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51572 entries, 0 to 51571
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Flight Type                   51572 non-null  object        
 1   Status                        51572 non-null  object        
 2   Departure IATA Code           51572 non-null  object        
 3   Departure ICAO Code           51572 non-null  object        
 4   Arrival IATA Code             51572 non-null  object        
 5   Arrival ICAO Code             51572 non-null  object        
 6   Arrival Scheduled Time        51572 non-null  datetime64[ns]
 7   Airline Name                  51572 non-null  object        
 8   Airline IATA Code             51572 non-null  object        
 9   Airline ICAO Code             51572 non-null  object        
 10  Flight Number                 51572 non-null  int64         
 11  Flight IATA Number          

In [949]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14910 entries, 0 to 14909
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Flight Type                   14910 non-null  object        
 1   Status                        14910 non-null  object        
 2   Departure IATA Code           14910 non-null  object        
 3   Departure ICAO Code           14910 non-null  object        
 4   Arrival IATA Code             14910 non-null  object        
 5   Arrival ICAO Code             14910 non-null  object        
 6   Arrival Scheduled Time        14910 non-null  datetime64[ns]
 7   Airline Name                  14910 non-null  object        
 8   Airline IATA Code             14910 non-null  object        
 9   Airline ICAO Code             14910 non-null  object        
 10  Flight Number                 14910 non-null  int64         
 11  Flight IATA Number          

In [950]:
cols = [col for col in train.columns if col != 'Delay'] + ['Delay']
train = train[cols]
test = test[cols]

In [951]:
train.columns

Index(['Flight Type', 'Status', 'Departure IATA Code', 'Departure ICAO Code',
       'Arrival IATA Code', 'Arrival ICAO Code', 'Arrival Scheduled Time',
       'Airline Name', 'Airline IATA Code', 'Airline ICAO Code',
       'Flight Number', 'Flight IATA Number', 'Flight ICAO Number',
       'Departure Scheduled Day', 'Departure Scheduled Date',
       'Departure Scheduled Hour', 'Departure Scheduled Year',
       'Departure Actual Runway Date', 'Departure Actual Runway Day',
       'Departure Actual Runway Hour', 'Departure Actual Runway Year',
       'Temperature (°F)', 'Dew Point (°F)', 'Humidity (%)',
       'Wind Speed (mph)', 'Pressure (in)', 'Precipitation (in)', 'Delay'],
      dtype='object')

In [952]:
test.columns

Index(['Flight Type', 'Status', 'Departure IATA Code', 'Departure ICAO Code',
       'Arrival IATA Code', 'Arrival ICAO Code', 'Arrival Scheduled Time',
       'Airline Name', 'Airline IATA Code', 'Airline ICAO Code',
       'Flight Number', 'Flight IATA Number', 'Flight ICAO Number',
       'Departure Scheduled Day', 'Departure Scheduled Date',
       'Departure Scheduled Hour', 'Departure Scheduled Year',
       'Departure Actual Runway Date', 'Departure Actual Runway Day',
       'Departure Actual Runway Hour', 'Departure Actual Runway Year',
       'Temperature (°F)', 'Dew Point (°F)', 'Humidity (%)',
       'Wind Speed (mph)', 'Pressure (in)', 'Precipitation (in)', 'Delay'],
      dtype='object')

In [953]:
train.to_csv('train_processed_data.csv', index=False)

In [954]:
test.to_csv('test_processed_data.csv', index=False)


In [538]:
#test_binary.to_csv('bintest_processed_data.csv', index=False)